In [1]:
!rm customers.sqlite customers_cleaned.csv

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import os

# Data Cleaning

In [3]:
# Path of the CSV file
csvfile = "customers.csv"

In [4]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv(csvfile, dtype=object)

In [5]:
# Preview DataFrame
# Note that some rows are missing gender
df.head()

,first_name,gender,id,last_name
0,David,Male,1,Jordan
1,Stephen,Male,2,NaN
2,Evelyn,Female,3,Grant
3,Joe,NaN,4,Mendoza
4,Benjamin,NaN,5,Rodriguez


In [6]:
# Use `dropna` to drop any rows where there is missing data
# Notice that index has not been reset and contains a gap (0,2,5,6,7)
df = df.dropna(axis=0)
df.head()

,first_name,gender,id,last_name
0,David,Male,1,Jordan
2,Evelyn,Female,3,Grant
5,Kathleen,Female,6,Crawford
6,Daniel,Male,7,Marshall
7,Diana,Female,8,Coleman


In [7]:
# Drop the 'gender' column and reset the index
# Note that after resetting the index, the gap is gone
df = df.drop(['gender'], axis=1).reset_index(drop=True)
df.head()

,first_name,id,last_name
0,David,1,Jordan
1,Evelyn,3,Grant
2,Kathleen,6,Crawford
3,Daniel,7,Marshall
4,Diana,8,Coleman


In [8]:
# Save the cleaned data to a file called `customers_cleaned.csv`
new_csv = "customers_cleaned.csv"
df.to_csv(new_csv, index=False)

# Database Creation

In [9]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [10]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///customers.sqlite")

In [11]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [12]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'

    id = Column(Integer, primary_key=True)
    first_name = Column(Text)
    last_name = Column(Text)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
# More on __repr__: https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr    

In [13]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [14]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv(new_csv)

In [15]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = new_df.to_dict(orient='records')

In [16]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

[{'first_name': 'David', 'id': 1, 'last_name': 'Jordan'}, {'first_name': 'Evelyn', 'id': 3, 'last_name': 'Grant'}, {'first_name': 'Kathleen', 'id': 6, 'last_name': 'Crawford'}, {'first_name': 'Daniel', 'id': 7, 'last_name': 'Marshall'}, {'first_name': 'Diana', 'id': 8, 'last_name': 'Coleman'}]


In [17]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [18]:
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table('customers', metadata, autoload=True)

In [19]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [20]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [21]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from customers limit 5").fetchall()

[(1, 'David', 'Jordan'),
 (3, 'Evelyn', 'Grant'),
 (6, 'Kathleen', 'Crawford'),
 (7, 'Daniel', 'Marshall'),
 (8, 'Diana', 'Coleman')]